In [1]:
import os
import shutil
import glob
import json
import random
import scipy.io as sio
import pandas as pd
import numpy as np

In [2]:
raw_data_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_rawfmri'

bids_path = '/Users/zeynepenkavi/Downloads/overtrained_decisions_bidsfmri'

subnums = ['601', '609', '611', '619', '621', '629']

sessions = ['01', '02', '03']

data_types = ['anat', 'func']

misc_files = ['dataset_description.json', 'participants.json', 'README', 'CHANGES']

In [3]:
from bidsify_helpers import bidsify_func_events

In [4]:
bidsify_func_events(raw_path_ = raw_data_path, bids_path_ = bids_path)

Processing: taskBC_subj609_day7.mat
Saving: sub-609/ses-02/func/sub-609_ses-02_task-binaryChoice_run-03_events.tsv
Processing: taskYN_subj609_day7_session4.mat
Saving: sub-609/ses-02/func/sub-609_ses-02_task-yesNo_run-02_events.tsv
Processing: taskYN_subj609_day7_session3.mat
Saving: sub-609/ses-02/func/sub-609_ses-02_task-yesNo_run-01_events.tsv
Processing: taskBC_subj609_day11.mat
Saving: sub-609/ses-03/func/sub-609_ses-03_task-binaryChoice_run-03_events.tsv
Processing: taskYN_subj609_day11_session4.mat
Saving: sub-609/ses-03/func/sub-609_ses-03_task-yesNo_run-02_events.tsv
Processing: taskYN_subj609_day11_session3.mat
Saving: sub-609/ses-03/func/sub-609_ses-03_task-yesNo_run-01_events.tsv
Processing: taskBC_subj609_day3.mat
Saving: sub-609/ses-01/func/sub-609_ses-01_task-binaryChoice_run-03_events.tsv
Processing: taskYN_subj609_day3_session4.mat
Saving: sub-609/ses-01/func/sub-609_ses-01_task-yesNo_run-02_events.tsv
Processing: taskYN_subj609_day3_session3.mat
Saving: sub-609/ses-01